In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("Rush Duel")

---

Table of Contents <a class="jp-toc-ignore"></a>
=================
* [1 Data aquisition](#data-aquisition)
  * [1.1 Fetch online data](#fetch-online-data)
  * [1.2 Merge data](#merge-data)
* [2 Check changes](#check-changes)
  * [2.1 Load previous data](#load-previous-data)
  * [2.2 Generate changelog](#generate-changelog)
  * [2.3 Save data](#save-data)
* [3 Data visualization](#data-visualization)
  * [3.1 Card type](#card-type)
  * [3.2 Monsters](#monsters)
    * [3.2.1 Attribute](#attribute)
    * [3.2.2 Primary type](#primary-type)
      * [3.2.2.1 Has effect discrimination](#has-effect-discrimination)
      * [3.2.2.2 By attribute](#by-attribute)
    * [3.2.3 Monster type](#monster-type)
      * [3.2.3.1 By Attribute](#by-attribute)
      * [3.2.3.2 By primary type](#by-primary-type)
    * [3.2.4 ATK](#atk)
    * [3.2.5 DEF](#def)
    * [3.2.6 Maximum mode](#maximum-mode)
      * [3.2.6.1 MAXIMUM ATK](#maximum-atk)
        * [3.2.6.1.1 By ATK](#by-atk)
        * [3.2.6.1.2 By DEF](#by-def)
    * [3.2.7 Level/Rank](#level/rank)
      * [3.2.7.1 ATK statistics](#atk-statistics)
      * [3.2.7.2 DEF statistics](#def-statistics)
  * [3.3 Spell & Trap](#spell-&-trap)
    * [3.3.1 Properties](#properties)
  * [3.4 Effect type](#effect-type)
    * [3.4.1 Card type discrimination](#card-type-discrimination)
  * [3.5 Archseries](#archseries)
    * [3.5.1 By card type](#by-card-type)
    * [3.5.2 By primary type](#by-primary-type)
    * [3.5.3 By monster type](#by-monster-type)
    * [3.5.4 By property](#by-property)
  * [3.6 Legend](#legend)
      * [3.6.0.1 By card type](#by-card-type)
      * [3.6.0.2 By Attribute](#by-attribute)
      * [3.6.0.3 By primary type](#by-primary-type)
      * [3.6.0.4 By monster type](#by-monster-type)
  * [3.7 Artworks](#artworks)
    * [3.7.1 By card type](#by-card-type)
    * [3.7.2 By primary type](#by-primary-type)
  * [3.8 Errata](#errata)
    * [3.8.1 By card type](#by-card-type)
    * [3.8.2 By primary type](#by-primary-type)
    * [3.8.3 By artwork](#by-artwork)
* [4 Epilogue](#epilogue)
  * [4.1 HTML export](#html-export)
  <!-- * [4.2 Git](#git) -->

# Data aquisition

## Fetch online data

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
# Fetch rush cards
rush_df = fetch_rush()

# Fetch errata
errata_df = fetch_errata()

## Merge data

In [ ]:
rush_df = merge_errata(rush_df, errata_df)
print("Data merged")

# Check changes

## Load previous data

In [ ]:
# Get latest file if exist
tuple_cols = [
    "Effect type",
    "Archseries",
    "Artwork",
    "Errata",
    "Legend",
    "Maximum mode",
]
previous_df, previous_ts = load_latest_data("rush", tuple_cols, return_ts=True)

if previous_df is not None:
    previous_df = previous_df.astype(rush_df[previous_df.columns.intersection(rush_df.columns)].dtypes.to_dict())

## Generate changelog

In [ ]:
if previous_df is None:
    changelog = None
    print("Skipped")
else:
    changelog = generate_changelog(previous_df, rush_df, col="Name")
    if not changelog.empty:
        display(changelog)
        new_names = changelog.index.value_counts()
        new_names = new_names[new_names == 1].index
        if len(new_names) > 0:
            print("New cards:")
            display(changelog.loc[new_names])
        changelog.to_csv(
            dirs.DATA / make_filename(report="rush", timestamp=timestamp, previous_timestamp=previous_ts),
            index=True,
        )
        print("Changelog saved")

## Save data

In [ ]:
if changelog is not None and changelog.empty:
    print("No changes. New data not saved")
else:
    rush_df.to_csv(
        dirs.DATA / make_filename(report="rush", timestamp=timestamp),
        index=False,
    )
    print("Data saved")

# Data visualization

In [ ]:
rush_df

## Card type

In [ ]:
print("Total number of card types:", rush_df["Card type"].nunique())

In [ ]:
card_type_colors = [plot.colors_dict[i] for i in rush_df["Card type"].value_counts().index]
rush_df["Card type"].value_counts().plot.bar(figsize=(14, 6), rot=0, color=card_type_colors)
plt.show()

## Monsters

### Attribute

In [ ]:
print("Total number of attributes:", rush_df["Attribute"].nunique())

In [ ]:
rush_df.drop(columns=["Card type"]).groupby("Attribute").nunique()

In [ ]:
attribute_colors = [plot.colors_dict[i] for i in rush_df["Attribute"].value_counts().index]
rush_df["Attribute"].value_counts().plot.bar(figsize=(16, 6), rot=0, color=attribute_colors)
plt.show()

### Primary type

In [ ]:
print("Total number of primary types:", rush_df["Primary type"].nunique())

In [ ]:
rush_df.drop(columns=["Card type"]).groupby("Primary type").nunique()

#### Has effect discrimination

In [ ]:
has_effect = rush_df["Primary type"].where(rush_df["Effect type"].notna()).value_counts().rename("Effect")
no_effect = rush_df["Primary type"].where(rush_df["Effect type"].isna()).value_counts().rename("No Effect")
effect = pd.concat([has_effect, no_effect], axis=1).fillna(0).astype(int)
effect

In [ ]:
monster_type_colors = {
    "No Effect": plot.colors_dict["Normal Monster"],
    "Effect": [plot.colors_dict[i] for i in effect.index],
}
effect.plot.bar(
    figsize=(12, 6),
    stacked=True,
    rot=0,
    legend=True,
    color=monster_type_colors,
    title="Primary types - Has effect",
)
# plt.yscale('log')
plt.show()

Obs: Effect monster can show as no effect if it is not released yet

#### By attribute

In [ ]:
primmary_crosstab = pd.crosstab(rush_df["Primary type"], rush_df["Attribute"])
primmary_crosstab

In [ ]:
plt.figure(figsize=(16, 6))
sns.heatmap(
    primmary_crosstab,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=plot.LogNorm(),
)
plt.show()

### Monster type

In [ ]:
print("Total number of monster types:", rush_df["Monster type"].nunique())

In [ ]:
rush_df.drop(columns=["Card type"]).groupby("Monster type").nunique()

In [ ]:
monster_type_colors = plot.colors_dict["Monster Card"]
rush_df["Monster type"].value_counts().plot.bar(figsize=(18, 6), rot=60, color=monster_type_colors)
plt.show()

#### By Attribute

In [ ]:
monster_crosstab = pd.crosstab(rush_df["Monster type"], rush_df["Attribute"], dropna=False)
monster_crosstab

In [ ]:
plt.figure(figsize=(18, 4))
sns.heatmap(
    monster_crosstab[monster_crosstab > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=plot.LogNorm(),
)
plt.show()

#### By primary type

In [ ]:
monster_crosstab_b = pd.crosstab(rush_df["Monster type"], rush_df["Primary type"], dropna=False)
monster_crosstab_b

In [ ]:
plt.figure(figsize=(18, 3))
sns.heatmap(
    monster_crosstab_b[monster_crosstab_b > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=plot.LogNorm(),
)
plt.show()

### ATK

In [ ]:
print("Total number of ATK values:", rush_df["ATK"].nunique())

In [ ]:
rush_df.drop(columns=["Card type"]).groupby("ATK").nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
atk_colors = plot.colors_dict["Monster Card"]
rush_df["DEF"].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce")).plot.bar(
    figsize=(18, 6), color=atk_colors
)
plt.show()

### DEF

In [ ]:
print("Total number of DEF values:", rush_df["DEF"].nunique())

In [ ]:
rush_df.drop(columns=["Card type"]).groupby("DEF").nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
def_colors = plot.colors_dict["Monster Card"]
rush_df["DEF"].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce")).plot.bar(
    figsize=(18, 6), color=def_colors
)
plt.show()

### Maximum mode

In [ ]:
print("Total cards requiring maximum mode:", rush_df["Maximum mode"].sum())

In [ ]:
rush_df[rush_df["Maximum mode"]]

#### MAXIMUM ATK

In [ ]:
print("Total number of MAXIMUM ATK values:", rush_df["MAXIMUM ATK"].nunique())

In [ ]:
def_colors = plot.colors_dict["Monster Card"]
rush_df["MAXIMUM ATK"].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce")).plot.bar(
    figsize=(10, 6), color=def_colors
)
plt.show()

##### By ATK

In [ ]:
max_atk_crosstab = pd.crosstab(rush_df["ATK"], rush_df["MAXIMUM ATK"])
max_atk_crosstab

##### By DEF

In [ ]:
max_def_crosstab = pd.crosstab(rush_df["DEF"], rush_df["MAXIMUM ATK"])
max_def_crosstab

### Level

In [ ]:
rush_df.drop(columns=["Card type"]).groupby("Level").nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
stars_colors = plot.colors_dict["Level"]
rush_df["Level"].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce")).plot.bar(
    figsize=(16, 6), rot=0, color=stars_colors
)
plt.show()

#### ATK statistics

In [ ]:
rush_df[["Level", "ATK"]].apply(pd.to_numeric, errors="coerce").dropna().astype(int).groupby("Level").describe().round(1)

#### DEF statistics

In [ ]:
rush_df[["Level", "DEF"]].apply(pd.to_numeric, errors="coerce").dropna().astype(int).groupby("Level").describe().round(1)

## Spell & Trap

### Properties

In [ ]:
print("Total number of properties:", rush_df["Property"].nunique())

In [ ]:
rush_df.drop(columns=["Card type"]).groupby("Property").nunique()

In [ ]:
st_colors = [plot.colors_dict[i] for i in rush_df[["Card type", "Property"]].value_counts().index.get_level_values(0)]
rush_df["Property"].value_counts().plot.bar(figsize=(10, 6), rot=0, color=st_colors)
plt.show()

## Effect type

In [ ]:
print("Total number of effect types:", rush_df["Effect type"].explode().nunique())

In [ ]:
rush_df.explode("Effect type").groupby("Effect type").nunique()

### Card type discrimination

In [ ]:
type_diff = rush_df[["Card type", "Effect type"]].explode("Effect type").value_counts().unstack(0).fillna(0).astype(int)
type_diff

In [ ]:
type_diff_colors = {type: plot.colors_dict[type] for type in rush_df["Card type"].unique()}
type_diff.plot.bar(figsize=(10, 6), stacked=True, rot=0, color=type_diff_colors)
plt.show()

## Archseries

In [ ]:
exploded_archseries = rush_df.explode("Archseries")
print("Total number of Archseries:", exploded_archseries["Archseries"].nunique())

In [ ]:
exploded_archseries.groupby("Archseries").nunique()

In [ ]:
exploded_archseries["Archseries"].value_counts().head(30).plot.barh(figsize=(10, 15), title="Top 30 Archtypes/Series")
plt.show()

### By card type

In [ ]:
archseries_crosstab = pd.crosstab(exploded_archseries["Archseries"], exploded_archseries["Card type"], margins=True)
archseries_crosstab

### By primary type

In [ ]:
archseries_crosstab_b = pd.crosstab(exploded_archseries["Archseries"], exploded_archseries["Primary type"], margins=True)
archseries_crosstab_b

### By monster type

In [ ]:
archseries_crosstab_d = pd.crosstab(exploded_archseries["Archseries"], exploded_archseries["Monster type"], margins=True)
archseries_crosstab_d

### By property

In [ ]:
archseries_crosstab_e = pd.crosstab(exploded_archseries["Archseries"], exploded_archseries["Property"], margins=True)
archseries_crosstab_e

## Legend

In [ ]:
print("Total number of Legend cards", rush_df["Legend"].sum())

In [ ]:
rush_df[rush_df["Legend"]]

#### By card type

In [ ]:
card_type_colors = [plot.colors_dict[i] for i in rush_df[rush_df["Legend"]]["Card type"].value_counts().index]
rush_df[rush_df["Legend"]]["Card type"].value_counts().plot.bar(figsize=(10, 6), rot=0, color=card_type_colors)
plt.show()

#### By Attribute

In [ ]:
attribute_colors = [plot.colors_dict[i] for i in rush_df[rush_df["Legend"]]["Attribute"].value_counts().index]
rush_df[rush_df["Legend"]]["Attribute"].value_counts().plot.bar(figsize=(12, 6), rot=0, color=attribute_colors)
plt.show()

#### By primary type

In [ ]:
monster_type_colors = [plot.colors_dict[i] for i in rush_df[rush_df["Legend"]]["Primary type"].value_counts().index]
rush_df[rush_df["Legend"]]["Primary type"].value_counts().plot.bar(
    figsize=(10, 6),
    stacked=True,
    rot=0,
    legend=True,
    color=monster_type_colors,
)
# plt.yscale('log')
plt.show()

#### By monster type

In [ ]:
monster_type_colors = plot.colors_dict["Monster Card"]
rush_df[rush_df["Legend"]]["Monster type"].value_counts().plot.bar(figsize=(16, 6), rot=90, color=monster_type_colors)
plt.show()

## Artworks

In [ ]:
print(
    "Total number of cards with edited or alternate artworks:",
    rush_df["Artwork"].count(),
)

In [ ]:
rush_df[["Name", "Artwork"]].dropna()

In [ ]:
artwork_value_counts = rush_df["Artwork"].value_counts()
plt.figure(figsize=(20, 8))
plt.title("Artworks")
plot.venn2(
    subsets=(
        artwork_value_counts[("Alternate",)],
        artwork_value_counts[("Edited",)],
        artwork_value_counts[("Alternate", "Edited")],
    ),
    set_labels=("Alternate artwork", "Edited artwork"),
)
plt.show()

### By card type

In [ ]:
artwork_crosstab = pd.crosstab(rush_df["Artwork"], rush_df["Card type"])
artwork_crosstab

### By primary type

In [ ]:
artwork_crosstab_b = pd.crosstab(rush_df["Artwork"], rush_df["Primary type"])
artwork_crosstab_b

More granularity is unnecessary

## Errata

In [ ]:
print("Total number of cards with errata:", rush_df["Errata"].count())

In [ ]:
rush_df[["Name", "Errata"]].dropna()

### By card type

In [ ]:
errata_crosstab = pd.crosstab(rush_df["Errata"], rush_df["Card type"])
errata_crosstab

### By primary type

In [ ]:
errata_crosstab_b = pd.crosstab(rush_df["Errata"], rush_df["Primary type"])
errata_crosstab_b

### By artwork

In [ ]:
errata_crosstab_c = pd.crosstab(rush_df["Artwork"], rush_df["Errata"])
errata_crosstab_c

More granularity is unnecessary

# Epilogue

In [ ]:
benchmark(report="rush", timestamp=timestamp)

In [ ]:
footer(timestamp)

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
export_notebook(dirs.NOTEBOOKS.user / "Rush.ipynb")

## Git

In [ ]:
git.commit("*[Rr]ush*", f"Rush duel update - {timestamp.isoformat()}")